<a href="https://colab.research.google.com/github/seopp/Tobigs19/blob/main/week3_DT_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DT Assignment

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


## Gini 계수를 구하는 함수 만들기

<img src="gini.png" width="200">

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

- 지니계수는 데이터의 통계적 분산 정도를 정량화 해서 표현한 값이다.
- 어떤 집합의 gini index가 높을수록 그 집단의 데이터가 분산되어 있음을 확인할 수 있다.

In [5]:
from functools import reduce

In [50]:
def get_gini(df, label):
    df_ = df.drop(columns=[label])
    
    uniq = df[label].unique()
    gini = 1
    for x in uniq:
        gini -= (df.query(f"{label} == '{x}'").shape[0]/df.shape[0])**2
    
    return gini


In [51]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

## Feature의 Class를 이진 분류로 만들기
- ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- Income 변수를 결과로 출력해주세요.

In [52]:
import itertools 

In [53]:
def get_binary_split(df, attribute):
    uniques = list(df[attribute].unique()) # 속성 데이터 고유값들을 담은 리스트 

    # 이중 For loop List Comprehension
    result = [
            list(item) 
            for i in range(1, len(uniques)) # 1부터 변수의 클래스 갯수-1 까지 Iteration
            for item in itertools.combinations(uniques, i) # i를 길이로 하는 조합 생성
        ]
    return result

In [54]:
get_binary_split(pd_data,'income')

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

## 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [55]:
def get_attribute_gini_index(df, attribute, label):
    result = {}
    binary_split = get_binary_split(df, attribute)
    n = len(df)
    for key in binary_split:
        t_index = df[attribute].map(lambda x: x in key) # Split한 클래스들에 속하는 df Index 추출
        Dj_len_1 = df[t_index].shape[0]
        Dj_len_2 = df[t_index.map(lambda x: not x)].shape[0]

        gini = (Dj_len_1 / n) * get_gini(df[t_index], label) + (Dj_len_2 / n) * get_gini(df[t_index.map(lambda x: not x)], label)

        key = ''.join(key)
        result[key] = gini

    return result

In [56]:
get_attribute_gini_index(pd_data, 'income', 'class_buys_computer')

{'high': 0.4428571428571429,
 'medium': 0.4583333333333333,
 'low': 0.45,
 'highmedium': 0.45,
 'highlow': 0.4583333333333333,
 'mediumlow': 0.4428571428571429}

- 여기서 가장 작은 Gini index값을 가지는 class를 확인합니다.

In [57]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())

('high', 0.4428571428571429)

In [58]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())[0]

'high'

## 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 모든 변수에 대한 Gini index(최소)를 출력해주세요.
- 해당 결과는 아래와 같이 나와야 합니다.

In [59]:
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]
# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])

# 각 변수를 대상으로 반복문 수행(해당 변수 중 가장 낮은 gini 계수와 변수 출력)
for feature in features:
    var, gini = min(get_attribute_gini_index(pd_data, feature, label).items())
    print(f"Minimum Gini index of {feature} : {gini}")


Minimum Gini index of age : 0.35714285714285715
Minimum Gini index of income : 0.4428571428571429
Minimum Gini index of student : 0.3673469387755103
Minimum Gini index of credit_rating : 0.42857142857142855


gini index가 가장 작게 나온 'age'를 가장 중요한 변수로 선정합니다.

이어서 해당 변수의 이진 분류된 각 class에 대해 Gini index도 계산합니다.

In [60]:
get_attribute_gini_index(pd_data, 'age', 'class_buys_computer')

{'youth': 0.3936507936507937,
 'middle_aged': 0.35714285714285715,
 'senior': 0.45714285714285713,
 'youthmiddle_aged': 0.45714285714285713,
 'youthsenior': 0.35714285714285715,
 'middle_agedsenior': 0.3936507936507937}

'age' 변수에서 gini index가 가장 작게 나온 'middle_aged' class를 선정합니다.

## Entropy 를 구하는 함수 만들기

<img src = https://miro.medium.com/max/1122/0*DkWdyGidNSfdT1Nu.png width = "350">

In [61]:
from math import log2

def getEntropy(df, feature) :
    
    
    """
    수식을 참고하여,
    데이터프레임 df에서 특정 feature에 대해 
    엔트로피를 구하는 함수를 작성해주세요.
    """
    
    prob = df[feature].value_counts().divide(df.shape[0])
    res = -sum(prob * np.log2(prob))
    return res


In [62]:
getEntropy(pd_data, "class_buys_computer")

0.9402859586706311

In [63]:
# 가장 중요한 변수로 선정된 목표변수를 제외한 다른 변수들에 대해
# 각 칼럼별로 엔트로피를 구해주는 함수를 작성해주세요.

def getGainA(df, feature) :
    result = {}
    ent_target = getEntropy(df, feature)
    
    columns = df.drop(columns=[feature]).columns
    for col in columns:
        entropy = 0
        for x in df[col].unique():
            w = df.query(f"{col} == '{x}'").shape[0]/df.shape[0]
            entropy += w * getEntropy(df.query(f"{col} == '{x}'"), feature)
        result[col] = ent_target - entropy
    return result


In [64]:
getGainA(pd_data, "class_buys_computer")

{'age': 0.24674981977443933,
 'income': 0.02922256565895487,
 'student': 0.15183550136234159,
 'credit_rating': 0.04812703040826949}